In [1]:
!pip install --quiet img2vec_pytorch

In [2]:
import pandas as pd

DIGITS = '/kaggle/input/handwritten-digits-from-0-to-9/Digits from 0 - 9.csv'
df = pd.read_csv(filepath_or_buffer=DIGITS)
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel12590,pixel12591,pixel12592,pixel12593,pixel12594,pixel12595,pixel12596,pixel12597,pixel12598,pixel12599
0,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [3]:
df.shape

(21553, 12601)

In [4]:
from plotly import express
express.histogram(data_frame=df, x='label')

In [5]:
from umap import UMAP
from arrow import now

columns = [column for column in df.columns if column.startswith('pixel')]
target = 'label'

time_start = now()
# running UMAP over all our data takes forever, and we should be able to get a sense
# of how strong a signal is in our data from a sample
sample_df = df.sample(n=5000, random_state=2024).copy()
reducer = UMAP(n_components=2, random_state=2024, transform_seed=2024, verbose=True, n_jobs=1, n_epochs=100)
sample_df[['x', 'y']] = pd.DataFrame(data=reducer.fit_transform(X=sample_df[columns]))
express.scatter(data_frame=sample_df, x='x', y='y', color=target, height=800, ).show()
print('UMAP done in {}'.format(now() - time_start))

2024-03-11 15:05:10.431985: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 15:05:10.432189: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 15:05:10.639182: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


UMAP(n_epochs=100, n_jobs=1, random_state=2024, transform_seed=2024, verbose=True)
Mon Mar 11 15:05:29 2024 Construct fuzzy simplicial set
Mon Mar 11 15:05:29 2024 Finding Nearest Neighbors
Mon Mar 11 15:05:29 2024 Building RP forest with 9 trees
Mon Mar 11 15:05:39 2024 NN descent for 12 iterations
	 1  /  12
	 2  /  12
	 3  /  12
	 4  /  12
	 5  /  12
	 6  /  12
	 7  /  12
	 8  /  12
	Stopping threshold met -- exiting after 8 iterations
Mon Mar 11 15:06:03 2024 Finished Nearest Neighbor Search
Mon Mar 11 15:06:08 2024 Construct embedding


Epochs completed:   0%|            0/100 [00:00]

	completed  0  /  100 epochs
	completed  10  /  100 epochs
	completed  20  /  100 epochs
	completed  30  /  100 epochs
	completed  40  /  100 epochs
	completed  50  /  100 epochs
	completed  60  /  100 epochs
	completed  70  /  100 epochs
	completed  80  /  100 epochs
	completed  90  /  100 epochs
Mon Mar 11 15:06:14 2024 Finished embedding


UMAP done in 0:00:48.336491


In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from arrow import now

time_start = now()
X_train, X_test, y_train, y_test = train_test_split(df[columns], df[target], test_size=0.2, random_state=2024, stratify=df[target])

regression = LogisticRegression(max_iter=100, tol=1e-6)
regression.fit(X=X_train, y=y_train)
print('fit complete after {} iterations.'.format(regression.n_iter_[0]))
print('accuracy: {:5.4f}'.format(regression.score(X=X_test, y=y_test)))
print('done in {}'.format(now() - time_start))

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



fit complete after 100 iterations.
accuracy: 0.1830
done in 0:01:47.343321


In [7]:
from arrow import now
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

MODEL = {
    'Naive Bayes': GaussianNB(),
#     'QDA': QuadraticDiscriminantAnalysis(),
#     '3 Nearest Neighbors': KNeighborsClassifier(n_neighbors=3),
#     '5 Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
    '7 Nearest Neighbors': KNeighborsClassifier(n_neighbors=7),
    '8 Nearest Neighbors': KNeighborsClassifier(n_neighbors=8),
    '9 Nearest Neighbors': KNeighborsClassifier(n_neighbors=9),
    '10 Nearest Neighbors': KNeighborsClassifier(n_neighbors=10),
#     'Linear SVM': SVC(kernel='linear', C=0.025, random_state=2024),
#     'Gaussian Process':GaussianProcessClassifier(1.0 * RBF(1.0), random_state=2024),
    'Decision Tree': DecisionTreeClassifier(max_depth=5, random_state=2024),
    '10 estimator Random Forest': RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1, random_state=2024),
    '20 estimator Random Forest': RandomForestClassifier(max_depth=5, n_estimators=20, max_features=1, random_state=2024),
    'Neural Net': MLPClassifier(alpha=1, max_iter=1000, random_state=2024),
    'AdaBoost': AdaBoostClassifier(algorithm='SAMME', random_state=2024),
    'RBF SVM': SVC(gamma=2, C=1, random_state=2024),
}

result = []
for name, clf in MODEL.items():
    time_start = now()
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    result.append((score, name))
    print('{:5.4f} {} {}'.format(score, now() - time_start, name))
result = sorted(result, key=lambda x: x[0], reverse=True)
print('best: {} {}'.format(result[0][0], result[0][1]))

0.1297 0:00:12.157021 Naive Bayes
0.1645 0:00:35.282455 7 Nearest Neighbors
0.1605 0:00:35.040745 8 Nearest Neighbors
0.1601 0:00:39.532800 9 Nearest Neighbors
0.1619 0:00:35.432488 10 Nearest Neighbors
0.1181 0:00:05.495916 Decision Tree
0.1169 0:00:01.313301 10 estimator Random Forest
0.1218 0:00:01.369648 20 estimator Random Forest
0.1037 0:11:16.153274 Neural Net
0.1116 0:01:49.421531 AdaBoost
0.1338 1:57:13.534734 RBF SVM
best: 0.16446300162375319 7 Nearest Neighbors
